# Lab 4 - Data

In [1]:
% matplotlib inline

The special command above will make all the `matplotlib` images appear in the notebook.

## Directions

**Failure to follow the directions will result in a "0"**

The due dates for each are indicated in the Syllabus and the course calendar. If anything is unclear, please email EN605.648@gmail.com the official email for the course or ask questions in the Lab discussion area on Blackboard.

The Labs also present technical material that augments the lectures and course notes.  You should read through the entire lab at the start of each module.

Please make sure you understand all the instructions.

### General Instructions

1.  You will be submitting your assignment to Blackboard.
    
This assignment has many files. You should include in your zip file:

    1. [jhed_id].ipynb - this file and your queries/reporting code.
    2. hurricanes.py - your preprocessing and data writing file.
    3. hurricanes.html - the local copy of the Wikipedia page.
    4. hurricanes.db - the SQLite database you create.
    5. hurricans.sql - the DDL commands to create your database.
    
Put all of these files in a directory/folder named [jhed_id] and zip it (only use Zip compression, nothing else). Submit your zip file to Blackboard.
    
2. Data Science is as much about what you write (communicating) as the code you execute (researching). In many places, you will be required to execute code and discuss both the purpose and the result. Additionally, Data Science is about reproducibility and transparency. This includes good communication with your team and possibly with yourself. Therefore, you must show **all** work.

3. Avail yourself of the Markdown/Codecell nature of the notebook. If you don't know about Markdown, look it up. Your notebooks should not look like ransom notes. Don't make everything bold. Clearly indicate what question you are answering.

4. Submit a cleanly executed notebook. The first code cell should say `In [1]` and each successive code cell should increase by 1 throughout the notebook.

## Individual Submission

## Getting and Storing Data

This Lab is about acquiring, cleaning and storing data (commonly known as "wrangling") as well as doing a little bit of analysis.

### Basic Outline

1. Use `curl` or `wget` obtain a local copy of the following web page: Atlantic Hurricane Season ( https://en.wikipedia.org/wiki/Atlantic_hurricane_season ). You can also just visit the page and save the HTML. **include this in your submission as `hurricanes.html`**.  This is important. In Spring 2016, the page was edited during the module and different people got different answers at different times.  You only need to be correct with respect to your `hurricanes.html` file.
2. create the SQLite3 database and initialize the tables. **include these files as `hurricanes.db` and `hurricanes.sql`**.  We are using SQLite3 not because it is the best database in the world but because its database files are shareable.
3. use Beautiful Soup 4 and Python, parse the HTML file into a usable dataset. Write the data to the database. **this code should be in a file `hurricanes.py` and included in your submission** Your code should have copious comments.
4. Write this data set to a SQLite3 database called `hurricanes.db` **include this in your submission**.
5. Run the requested queries against the data set. **see below** The results should be **nicely formatted**.

Although Wikipedia has an API, I do not what you to use it for this assignment.

## Learning Objectives

This assignment has several learning objectives:

* create a database and tables in one database type, SQLite
* use BeautifulSoup to parse a web page.
* use Regular Expressions to parse text to extract numbers.
* write data to a database using SQL.
* read data from a database using SQL.

You can reference the Module links to various tutorials on these topics.

### Details

The data is contained in many separate HTML tables. You will need to look at the raw HTML to determine how everything is laid out. Make sure you decompose the problem into nice, neat, focused functions. You should have a function that processes a table, one that processes a row, one that processes something that may be numeric.

You only need to get the data from the tables starting in the 1850s but note that not all years have the same data. You only need to save the following columns. The name is parentheses is the name the column should have in the database table.

- Year (`year`)
- Number of tropical storms (`tropical_storms`)
- Number of hurricanes (`hurricanes`)
- Number of Major Hurricanes (`major_hurricanes`)
- Deaths (`deaths`)
- Damage (`damage`)
- Notes (`notes`)

Note that "Damage" doesn't start until 1900s and "Notes" was added in 1880s. "Strongest Storm" should be added to the Notes column (even in years that didn't have Notes) as should "Retired Storms". The name of the database table should be `atlantic_hurricanes`. The name of the table file (SQLite3 uses a file) should be `hurricanes.db` (who knows...you might need to add Pacific storms someday).

There are a number of parsing problems which will most likely require regular expressions. First, the Deaths column has numbers that include commas and entries that are not numbers (Unknown and None). How should you code Unknown and None so that answers are not misleading but queries are still fairly straightforward to write?

Similarly, Damages has numbers with commas, currency signs and different amount words (millions, billions). How will you normalize all of these so that a query can compare them? You may need regular expressions.

Additionally, the way that Tropical Storms are accounted for seems to change mysteriously. Looking over the data, it's not immediately apparent when the interpretation should change. 1850s, 1860s definitely but 1870s? Not sure. It could just be a coincidence that there were never more hurricanes than tropical storms which seems to be the norm but see, for example, 1975. Welcome to Data Science!

You should put your parsing code in `hurricanes.py`. None of it should be in this file. Imagine this file is going to be the report you give to your boss.

## Documentation

Any time you run into a problem where you have to decide what to do--how to solve the problem or treat the values--document it here.

I had to decide how to handle the multiple tables. I ended up trying to read in all the tables by their trbody tag, and then iterating through the rows and storing it in a list. I turned it into a DataFrame just so I can clean it, and then I put it into a sql implementation.

There turned out to be another table at the bottom using the same class, so I dropped that manually. 

Tables are of different column sizes. I can't figure out how to fix this.

Once I get the tables in the right size, I also need to learn how to use regex to convert numbers with commas into actual numbers that Python can do comparisons on.

## Hurricanes.db

What is the *function* of `hurricanes.db` in this assignment?

It's a sql database that we can pull from using sql statements instead of python commands

### Queries

When you are done, you must write and execute the following queries against your database. Those queries should be run from this notebook. Find the documentation for using SQLite3 from Python (the library is already included). You should never output raw Python data structures instead, you need to output well-formatted tables. You may need to look around for a library to help you or write your own formatting code. `Pandas` is one possibility. 

However, I want you to use raw SQL for your queries so if you use `Pandas` use it only for the formatting of query results (**don't load the data into Pandas and use Pandas/Python to query the data**). Imagine that you were somewhere you couldn't get Pandas...you'd need to know how to do this without it.

**Write the most general query possible. Never assume that you are going to get only one result back (i.e, don't assume there won't be ties).** The query result should be in a nicely formatted table; don't show raw data structures to your boss or manager.

Additionally, don't just run the query. Having gotten an answer, add a textual description of the result to the question. Data Science is not about running code; code is a means to an end. The end is the communication of results. We never just run code in this class.

In [75]:
# imports
import requests
import sqlite3
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from sqlalchemy import create_engine

1\. For the 1920s, list the years by number of tropical storms, then hurricanes.

In [100]:
c.execute("Select * from atlantic_hurricanes where year between 1920 and 1930 order by num_tropical_storms,num_hurricanes")
pd.DataFrame(c.fetchall())

,0,1,2,3,4,5,6,7,8,9,10
0,83,1925,4,2,0,59+,$19.9 million,One,None,None,None
1,89,1930,3,2,2,"8,000",$50 million,"""Dominican Republic""",• Dominican Republic Hurricane of 1930 (cat 4),The fifth deadliest hurricane on record,None
2,80,1922,5,3,1,Unknown,Unknown,Two,None,None,None
3,87,1929,5,3,1,51,$9.0 million,"""Florida""",• Great Bahamas Hurricane of 1929 (cat 4),None,None
4,78,1920,5,4,0,2,$15.75 million,One,None,None,None
5,81,1923,9,4,1,0,Unknown,Five,None,None,None
6,85,1927,8,4,1,184,Unknown,"""Nova Scotia""",• Nova Scotia Hurricane of 1927 (cat 3),None,None
7,86,1928,6,4,1,"4,000+",$952.5+ million,"""Okeechobee""",• Great Okeechobee Hurricane of 1928 (cat 5),The Okeechobee hurricane is the only known hur...,None
8,79,1921,7,5,2,6,$36.5 million,"""Tampa Bay""",• Hurricane San Pedro of 1921 (cat 3) • Great...,None,None
9,82,1924,11,5,2,150+,Unknown,"""Cuba""",• Great Cuba Hurricane of 1924 (cat 5),The earliest officially classified Category 5 ...,None


2\. What year had the most tropical storms?

In [103]:
c.execute("Select year from atlantic_hurricanes where num_tropical_storms = (select max(num_tropical_storms) from hurricanes)")
c.fetchall()

[('1880',), ('1955',), ('2009',)]

3\. What year had the most major hurricanes?

In [46]:
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele]) # Get rid of empty values

In [107]:
c.execute("Select year from atlantic_hurricanes where num_major_hurricanes = (select max(num_major_hurricanes) from hurricanes)")
c.fetchall()

[('1919',)]

4\. What year had the most deaths?

In [108]:
c.execute("Select year from atlantic_hurricanes where deaths = (select max(deaths) from hurricanes)")
c.fetchall()

[('1902',),
 ('1905',),
 ('1907',),
 ('1908',),
 ('1914',),
 ('1922',),
 ('1923',),
 ('1924',),
 ('1927',),
 ('1937',),
 ('1939',)]

5\. What year had the most damage (not inflation adjusted)?

In [110]:
c.execute("Select year from atlantic_hurricanes where damage_usd = (select max(damage_usd) from hurricanes)")
c.fetchall()

[('2018',)]

6\. What year had the highest proportion of tropical storms turn into major hurricanes?

In [111]:
c.execute("Select year from atlantic_hurricanes where (num_tropical_storms/num_hurricanes) = (select max((num_tropical_storms/num_hurricanes)) from hurricanes)")
c.fetchall()

[('1878',)]

## Things to think about

1. What is the granularity of this data? (Are the rows the most specific observation possible?)
2. What if this data were contained in worksheets in an Excel file? Find a Python library or libraries that work with Excel spreadsheets.
3. Each section links to details about each hurricane season. Review each Season's page and discuss strategies for extracting the information for every hurricane.
4. Hurricane tracking maps were recently added. How would you get and store those images?
5. Damages are not inflation adjusted. How would you go about *enriching* your data with inflation adjusted dollars? Where should this additional data be stored and how would it be used?

1. The rows are not the most specific observations possible. We can get more specific by looking at everything on a monthly or even daily basis. 
2. This would be so much easier (for me at least, since I'm an excel junkie). I believe Pandas already has a read_excel function that imports excel data as a dataframe, so it would be pretty simple to read everything after it's become a dataframe.
3. Each page is separated into different hurricane numbers. We could potentially  extract information from the text of each subsection using Regex.
4. The maps could be stored as arrays, and the paths could be analyzed.
5. I guess the best way of getting inflation adjusted dollars is from the Bureau of Labor Statistics, and their Consumer Price Index. This would be the most common way of indexing inflation relative to common goods that everyone buys. 